In [43]:
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import torch

# Importing libraries
import copy
import os

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Subset
from typing import Tuple, List
from torch.optim import Optimizer

from nltk.corpus import words
import nltk

In [44]:
!pip install emoji

In [45]:
import emoji

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import os

BASE_PATH = 'drive/MyDrive/NLP_project/'
# BASE_PATH = './'
if not os.path.exists(BASE_PATH):
    raise ValueError('path does not exist')

In [48]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [49]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


# Load dataset

In [50]:
df_train = pd.read_csv(BASE_PATH + 'datasets/EmoTrain.csv')
df_test = pd.read_csv(BASE_PATH + 'datasets/EmoVal.csv')

In [51]:
df_train = df_train.drop(axis=1, columns=['Unnamed: 0'])
df_test = df_test.drop(axis=1, columns=['Unnamed: 0'])
df_train

,text,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
0,Is there some scripture you could quote me? I'...,1,0,0,0,0,1,0,0,0,0
1,Good. Now we just need people to dislike commi...,1,0,0,0,0,0,0,0,0,0
2,This was driving me NUTS!,0,1,0,0,0,0,0,0,0,0
3,Thank you for your advice!,0,0,0,0,0,0,0,0,0,1
4,Some do. Some don't. Blanket generalizations a...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
190097,They needed to insert the phrase “over mainten...,0,0,0,0,1,0,0,0,0,0
190098,Back in the seventies and eighties we all did ...,1,0,0,0,0,0,0,0,0,0
190099,"6lbs is a lap dog, if someone shoots that caus...",0,0,0,0,0,0,0,1,0,0
190100,This gets much worse on the 2nd loop.,0,0,0,0,1,0,0,0,0,0


In [52]:
step = 5_000
df_train = df_train[:step * 2]
df_test = df_test[:step * 2]

# Pre processing

## Description of data

In [53]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         10000 non-null  object
 1   praise       10000 non-null  int64 
 2   amusement    10000 non-null  int64 
 3   anger        10000 non-null  int64 
 4   disapproval  10000 non-null  int64 
 5   confusion    10000 non-null  int64 
 6   interest     10000 non-null  int64 
 7   sadness      10000 non-null  int64 
 8   fear         10000 non-null  int64 
 9   joy          10000 non-null  int64 
 10  love         10000 non-null  int64 
dtypes: int64(10), object(1)
memory usage: 859.5+ KB


In [54]:
print(df_train.isnull().sum())

text           0
praise         0
amusement      0
anger          0
disapproval    0
confusion      0
interest       0
sadness        0
fear           0
joy            0
love           0
dtype: int64


In [55]:
def split_features_labels(df) -> Tuple[pd.Series, pd.DataFrame]:
    x = df['text']
    y = df.drop(axis=1, columns=['text'])
    return x, y


x_train, y_train = split_features_labels(df_train)
x_test, y_test = split_features_labels(df_test)

In [56]:
x_train

,text
0,Is there some scripture you could quote me? I'...
1,Good. Now we just need people to dislike commi...
2,This was driving me NUTS!
3,Thank you for your advice!
4,Some do. Some don't. Blanket generalizations a...
...,...
9995,I wanna see her debate [NAME]. He would shit a...
9996,"Vegetarian, liked animals too. Lets just ignor..."
9997,That moment is always noted in YouTube video c...
9998,"“NPCs”, “cucks”, “blue pillers” Well, there’s ..."


In [57]:
y_train

,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
0,1,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1
4,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,1,0,0,0,0,0
9996,1,0,0,0,0,0,0,0,0,1
9997,0,0,0,0,1,0,0,0,0,0
9998,0,0,0,0,1,0,0,0,0,0


## Data cleaning

Lower case all the words

In [58]:
def lower_case(x):
    x = x.str.lower()
    return x

x_train = lower_case(x_train)
x_test = lower_case(x_test)

In [59]:
x_train

,text
0,is there some scripture you could quote me? i'...
1,good. now we just need people to dislike commi...
2,this was driving me nuts!
3,thank you for your advice!
4,some do. some don't. blanket generalizations a...
...,...
9995,i wanna see her debate [name]. he would shit a...
9996,"vegetarian, liked animals too. lets just ignor..."
9997,that moment is always noted in youtube video c...
9998,"“npcs”, “cucks”, “blue pillers” well, there’s ..."


Convert to tokens

In [60]:
x_train = x_train.to_list()
x_test = x_test.to_list()

In [61]:
print(len(x_train))
print(x_train[:5])

10000
["is there some scripture you could quote me? i'd like to read up on it just to be sure for myself", 'good. now we just need people to dislike commies more than they do now', 'this was driving me nuts!', 'thank you for your advice!', "some do. some don't. blanket generalizations are almost always false and unhelpful."]


In [62]:
def list_of_words(x):
    x = list(map(lambda i: i.split(), x))
    return x

x_train = list_of_words(x_train)
x_test = list_of_words(x_test)

Then we have **list** of **list** of **tokens**

In [63]:
x_train

[['is',
  'there',
  'some',
  'scripture',
  'you',
  'could',
  'quote',
  'me?',
  "i'd",
  'like',
  'to',
  'read',
  'up',
  'on',
  'it',
  'just',
  'to',
  'be',
  'sure',
  'for',
  'myself'],
 ['good.',
  'now',
  'we',
  'just',
  'need',
  'people',
  'to',
  'dislike',
  'commies',
  'more',
  'than',
  'they',
  'do',
  'now'],
 ['this', 'was', 'driving', 'me', 'nuts!'],
 ['thank', 'you', 'for', 'your', 'advice!'],
 ['some',
  'do.',
  'some',
  "don't.",
  'blanket',
  'generalizations',
  'are',
  'almost',
  'always',
  'false',
  'and',
  'unhelpful.'],
 ['those', 'are', 'separate', 'issues', 'from', 'the', 'sample', 'size.'],
 ['my',
  'fur',
  'son',
  'was',
  'the',
  'first',
  'i',
  'came',
  'out',
  'to.',
  'most',
  'important',
  'also'],
 ['really',
  'appreciate',
  'this',
  'post',
  'and',
  'article.',
  'i',
  'just',
  'subscribed',
  'and',
  'between',
  'this',
  'and',
  'the',
  'sidebar',
  "i'm",
  'feeling',
  'pumped',
  'about',
  'start

What kind of **characters** are there in the dataset?

In [64]:
characters = {'isalnum': 0}
for text in x_train:
    for word in text:
        for c in word:
            if c.isalnum():
                characters['isalnum'] += 1
            elif c in characters:
                characters[c] += 1
            else:
                characters[c] = 0

In [65]:
print(len(characters))
keys = list(characters.keys())
keys.sort()
print(keys)
characters

146
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', ']', '^', '_', 'isalnum', '~', '£', '¦', '©', '¯', '°', '͜', '͡', '\u200d', '–', '—', '‘', '’', '“', '”', '…', '€', '™', '≠', '☁', '☹', '☺', '♀', '♂', '♡', '♥', '❣', '❤', '️', '🅰', '🅱', '🇸', '🇺', '🍁', '🍑', '🍺', '🍻', '🎵', '🎶', '🏭', '🏻', '🏼', '🏽', '🏾', '🐇', '🐢', '👅', '👌', '👍', '👏', '👑', '👲', '👹', '💀', '💃', '💕', '💚', '💜', '💨', '💩', '💪', '💯', '📰', '🔥', '🕺', '😀', '😁', '😂', '😃', '😄', '😅', '😆', '😇', '😉', '😊', '😍', '😎', '😏', '😐', '😒', '😓', '😔', '😕', '😗', '😜', '😞', '😟', '😡', '😢', '😣', '😤', '😥', '😦', '😪', '😫', '😬', '😭', '😮', '😰', '😱', '😳', '😴', '😹', '🙃', '🙄', '🙈', '🙌', '🙏', '🤔', '🤕', '🤙', '🤢', '🤣', '🤦', '🤬', '🤮', '🤷', '🦀', '🧛']


{'isalnum': 546413,
 '?': 1248,
 "'": 3386,
 '.': 11095,
 '!': 1966,
 '’': 1615,
 '[': 1826,
 ']': 1826,
 ',': 3349,
 '"': 630,
 ':': 358,
 ')': 282,
 '-': 456,
 '💩': 0,
 '/': 280,
 '*': 439,
 '(': 206,
 '^': 50,
 '>': 167,
 '~': 69,
 '”': 123,
 '😒': 2,
 '£': 3,
 '“': 119,
 '🙄': 5,
 '🍻': 0,
 '—': 4,
 '😂': 63,
 '=': 21,
 '$': 43,
 '😍': 3,
 '😪': 2,
 ';': 34,
 '&': 15,
 '😭': 7,
 '❤': 12,
 '️': 42,
 '%': 45,
 '͡': 5,
 '͜': 2,
 '🅱': 13,
 '😔': 2,
 '+': 28,
 '🤕': 1,
 '@': 7,
 '🤣': 11,
 '#': 39,
 '🙌': 3,
 '😇': 1,
 '🎶': 1,
 '💃': 3,
 '<': 26,
 '\u200d': 14,
 '♀': 10,
 '☺': 7,
 '😢': 3,
 '😦': 0,
 '😜': 2,
 '♡': 0,
 '_': 24,
 '‘': 18,
 '💪': 2,
 '🤔': 10,
 '😣': 1,
 '😊': 7,
 '🐢': 0,
 '😄': 1,
 '😐': 1,
 '😤': 58,
 '😞': 1,
 '¦': 0,
 '¯': 7,
 '😱': 0,
 '👍': 6,
 '🍑': 1,
 '😹': 0,
 '👌': 5,
 '🏾': 2,
 '😕': 1,
 '♂': 2,
 '🏻': 6,
 '–': 0,
 '💕': 4,
 '❣': 0,
 '😎': 8,
 '😡': 5,
 '🤬': 3,
 '😴': 1,
 '€': 1,
 '👏': 27,
 '™': 1,
 '🤢': 2,
 '🕺': 0,
 '🙃': 4,
 '🔥': 5,
 '😀': 1,
 '🅰': 0,
 '🤦': 1,
 '🏼': 0,
 '😅': 2,
 '😮': 1,
 '😆': 1,

- Remove non-semantic characters
- Convert `?` and `!` and `emojis` to tokens

In [66]:
def split_special_chars(word: str):
    if word.isalnum():
        return word, []
    specials = list(filter(lambda c: not c.isalnum(), word))
    return ''.join(filter(lambda c: c.isalnum(), word)), specials

def clean_tokens(x):
    for text in tqdm(x):
        specials = []
        for i, word in enumerate(text):
            text[i], special_word = split_special_chars(word)
            specials.extend(special_word)
        text.extend(specials)

clean_tokens(x_train)
clean_tokens(x_test)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [67]:
x_train

[['is',
  'there',
  'some',
  'scripture',
  'you',
  'could',
  'quote',
  'me',
  'id',
  'like',
  'to',
  'read',
  'up',
  'on',
  'it',
  'just',
  'to',
  'be',
  'sure',
  'for',
  'myself',
  '?',
  "'"],
 ['good',
  'now',
  'we',
  'just',
  'need',
  'people',
  'to',
  'dislike',
  'commies',
  'more',
  'than',
  'they',
  'do',
  'now',
  '.'],
 ['this', 'was', 'driving', 'me', 'nuts', '!'],
 ['thank', 'you', 'for', 'your', 'advice', '!'],
 ['some',
  'do',
  'some',
  'dont',
  'blanket',
  'generalizations',
  'are',
  'almost',
  'always',
  'false',
  'and',
  'unhelpful',
  '.',
  "'",
  '.',
  '.'],
 ['those', 'are', 'separate', 'issues', 'from', 'the', 'sample', 'size', '.'],
 ['my',
  'fur',
  'son',
  'was',
  'the',
  'first',
  'i',
  'came',
  'out',
  'to',
  'most',
  'important',
  'also',
  '.'],
 ['really',
  'appreciate',
  'this',
  'post',
  'and',
  'article',
  'i',
  'just',
  'subscribed',
  'and',
  'between',
  'this',
  'and',
  'the',
  'side

Remove stop-words

In [68]:
print(len(ENGLISH_STOP_WORDS))
print(ENGLISH_STOP_WORDS)

318
frozenset({'sometimes', 'might', 'take', 'yourselves', 'something', 'seeming', 'de', 'ours', 'only', 'of', 'with', 'cannot', 'to', 'sixty', 'be', 'there', 'where', 'every', 'cry', 'should', 'besides', 'once', 'behind', 'are', 'any', 'find', 'hence', 'we', 'formerly', 'someone', 'therefore', 'off', 'across', 'whereafter', 'than', 'none', 'already', 'anything', 'beforehand', 'would', 'became', 'them', 'please', 'seems', 'thus', 'thru', 'thereafter', 'within', 'next', 'one', 'move', 'whereby', 'front', 'beyond', 'forty', 'part', 'amount', 'at', 'co', 'noone', 'our', 'somehow', 'on', 'ie', 'latter', 'because', 'becoming', 'rather', 'beside', 'had', 'which', 'myself', 'whom', 'hereupon', 'their', 'anyone', 'together', 'before', 'hereby', 'onto', 'often', 'i', 'even', 'why', 'after', 'nowhere', 'up', 'un', 'go', 'two', 'get', 'else', 'toward', 'can', 'whose', 'many', 'whereupon', 'alone', 'very', 'due', 'see', 'less', 'via', 'nobody', 'since', 'made', 'hasnt', 'out', 'being', 'etc', 'mos

# Vectorize the dataset

An example of vectorization

In [69]:
df = pd.DataFrame(dtype=bool)
for i, text in enumerate(tqdm(x_train[:5])):
    for word in text:
        df.loc[i, word] = True
df.fillna(False, inplace=True)

  0%|          | 0/5 [00:00<?, ?it/s]

<ipython-input-69-c6cb43738d2c>:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(False, inplace=True)


In [70]:
df

,is,there,some,scripture,you,could,quote,me,id,like,...,advice,dont,blanket,generalizations,are,almost,always,false,and,unhelpful
0,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,...,False,True,True,True,True,True,True,True,True,True


## Use English word set

In [71]:
nltk.download('words')  # Download the word corpus if not already downloaded
word_list = words.words()  # Get the list of words
print(len(word_list))  # Count number of words
print(list(word_list)[:10])  # Print first 10 words

236736
['A', 'a', 'aa', 'aal', 'aalii', 'aam', 'Aani', 'aardvark', 'aardwolf', 'Aaron']


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


Clean the word set

In [72]:
word_list = set(map(lambda x: x.lower(), words.words())) - ENGLISH_STOP_WORDS
word_list.add('!')
word_list.add('?')
print(len(word_list))

234076


Get emojis

In [73]:
# Get a dictionary of all emoji aliases and their Unicode
emoji_set = emoji.EMOJI_DATA.keys()
print(len(emoji_set))  # Number of emojis

5042


In [74]:
word_list = list(word_list.union(emoji_set))
word_list.sort()
print(len(word_list))

239118


Create an index for every unique word

In [75]:
tensor_index = {}
for i, word in enumerate(tqdm(word_list)):
    tensor_index[word] = i

  0%|          | 0/239118 [00:00<?, ?it/s]

In [76]:
print(len(tensor_index))

239118


Create a tensor

In [77]:
def create_x_tensor(x):
    x_tensor = torch.zeros(len(x), len(tensor_index), dtype=torch.float16)
    for i, text in enumerate(tqdm(x)):
        for word in text:
            if word in tensor_index:
                x_tensor[i, tensor_index[word]] += 1
    return x_tensor

In [78]:
# x_train
last_index = 0
for i in range(0, len(x_train), step):
    last_index = i
    torch.save(create_x_tensor(x_train[i:i+step]), BASE_PATH + f'tensors/x_train_tensor{i//step}.pt')
# torch.save(create_x_tensor(x_train[last_index:]), BASE_PATH + f'tensors/x_train_tensor{last_index//step + 1}.pt')

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [79]:
# x_test
torch.save(create_x_tensor(x_test), BASE_PATH + 'tensors/x_test_tensor.pt')

  0%|          | 0/10000 [00:00<?, ?it/s]

Convert labels to tensor

In [80]:
y_train.describe()

,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
count,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000
mean,0.157800,0.065700,0.09650,0.078300,0.332900,0.065500,0.078400,0.020900,0.12960,0.116100
std,0.364572,0.247769,0.29529,0.268657,0.471275,0.247418,0.268813,0.143057,0.33588,0.320361
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,0.000000,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [81]:
def create_y_tensor(y):
    y_tensor = torch.tensor(y.values, dtype=torch.float16)
    return y_tensor

In [82]:
# y_train
last_index = 0
for i in range(0, len(y_train), step):
    last_index = i
    torch.save(create_y_tensor(y_train[i:i+step]), BASE_PATH + f'tensors/y_train_tensor{i//step}.pt')
# torch.save(create_y_tensor(y_train[last_index:]), BASE_PATH + f'tensors/y_train_tensor{last_index//step + 1}.pt')

In [83]:
torch.save(create_y_tensor(y_test), BASE_PATH + 'tensors/y_test_tensor.pt')